### ライブラリ・加工用データ準備

In [1]:
import pandas as pd
import re

In [2]:
DATA_PREFECTURE = """
京都府
北海道
青森県
岩手県
宮城県
秋田県
山形県
福島県
茨城県
栃木県
群馬県
埼玉県
千葉県
東京都
神奈川県
新潟県
富山県
石川県
福井県
山梨県
長野県
岐阜県
静岡県
愛知県
三重県
滋賀県
大阪府
兵庫県
奈良県
和歌山県
鳥取県
島根県
岡山県
広島県
山口県
徳島県
香川県
愛媛県
高知県
福岡県
佐賀県
長崎県
熊本県
大分県
宮崎県
鹿児島県
沖縄県
"""

prefectures = re.findall(r'(\w+)\n',DATA_PREFECTURE)

hankaku_table = str.maketrans({
    '1': '１',
    '2': '２',
    '3': '３',
    '4': '４',
    '5': '５',
    '6': '６',
    '7': '７',
    '8': '８',
    '9': '９',
    '0': '０',
})
zenkaku_table = str.maketrans({
    '１': '1',
    '２': '2',
    '３': '3',
    '４': '4',
    '５': '5',
    '６': '6',
    '７': '7',
    '８': '8',
    '９': '9',
    '０': '0',
})


### 読み込み

In [3]:
data_og = pd.read_csv('output.csv')
data = data_og.copy()
print(data.shape)
data.head()

(9913, 4)


,date,title,title_detail,url
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...


### 不審者の概要を抽出

In [4]:
# regex_parをget_inside_parに渡すと（）の中身が取得できる
regex_par = re.compile('（.+\）')

def get_inside_par(s, regex):
    return regex.search(s).group().strip('（）')

In [5]:
# title_detailから不審者の概要を抽出
data['incident'] = data['title'].apply(get_inside_par, regex = regex_par)
data.head()

,date,title,title_detail,url,incident
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行


### 住所と日付を分離

In [6]:
# title_detailの前半と後半を分離する
data['content'] = data['title_detail'].apply(lambda x: x.split('　')[0])
data['datetime'] = data['title_detail'].apply(lambda x: x.split('　')[1])
data.head()

,date,title,title_detail,url,incident,content,datetime
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日［解決］


### 日付の加工

In [7]:
# datetimeの[解決]を消す
data['datetime'] = data['datetime'].apply(lambda x: x.replace('［解決］', ''))
data.head()

,date,title,title_detail,url,incident,content,datetime
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日


In [8]:
# 日付がないレコードを捨てる
data = data[data['datetime'].apply(lambda s: '日' in s)]
data.shape

(9904, 7)

In [9]:
# datetimeに含まれる「からX月Y日」を削除
kara_flag = data['datetime'].apply(lambda s: 'から' in s)
data[kara_flag].head()

,date,title,title_detail,url,incident,content,datetime
61,2024/04/30,神奈川県 茅ケ崎市（バイク盗）,（神奈川）茅ケ崎市中島でバイク盗 ４月２７日から２９日,http://this.kiji.is/1158036866734457566?c=1347...,バイク盗,（神奈川）茅ケ崎市中島でバイク盗,４月２７日から２９日
95,2024/05/31,埼玉県 さいたま市大宮区（自動車盗）,（埼玉）さいたま市大宮区三橋４丁目付近で自動車盗 ５月２９日から３０日,http://this.kiji.is/1169265874224071455?c=1347...,自動車盗,（埼玉）さいたま市大宮区三橋４丁目付近で自動車盗,５月２９日から３０日
101,2024/05/31,埼玉県 本庄市（自動車盗）,（埼玉）本庄市西富田で自動車盗 ５月２９日から３０日,http://this.kiji.is/1169265316081402294?c=1347...,自動車盗,（埼玉）本庄市西富田で自動車盗,５月２９日から３０日
107,2024/05/31,神奈川県 横浜市青葉区（自動車破壊）,（神奈川）横浜市青葉区奈良町で自動車破壊 ５月２８日から２９日,http://this.kiji.is/1169264637737632561?c=1347...,自動車破壊,（神奈川）横浜市青葉区奈良町で自動車破壊,５月２８日から２９日
277,2024/04/30,千葉県 松戸市（部品狙い）,（千葉）松戸市根本で部品狙い ４月２６日から２９日,http://this.kiji.is/1157892739379396851?c=1347...,部品狙い,（千葉）松戸市根本で部品狙い,４月２６日から２９日


In [10]:
# datetimeに含まれる「からX月Y日」を削除
remove_kara = lambda s: re.sub(r'から.*', '', s)
data['datetime'] = data['datetime'].apply(remove_kara)

In [11]:
data[kara_flag].head()

,date,title,title_detail,url,incident,content,datetime
61,2024/04/30,神奈川県 茅ケ崎市（バイク盗）,（神奈川）茅ケ崎市中島でバイク盗 ４月２７日から２９日,http://this.kiji.is/1158036866734457566?c=1347...,バイク盗,（神奈川）茅ケ崎市中島でバイク盗,４月２７日
95,2024/05/31,埼玉県 さいたま市大宮区（自動車盗）,（埼玉）さいたま市大宮区三橋４丁目付近で自動車盗 ５月２９日から３０日,http://this.kiji.is/1169265874224071455?c=1347...,自動車盗,（埼玉）さいたま市大宮区三橋４丁目付近で自動車盗,５月２９日
101,2024/05/31,埼玉県 本庄市（自動車盗）,（埼玉）本庄市西富田で自動車盗 ５月２９日から３０日,http://this.kiji.is/1169265316081402294?c=1347...,自動車盗,（埼玉）本庄市西富田で自動車盗,５月２９日
107,2024/05/31,神奈川県 横浜市青葉区（自動車破壊）,（神奈川）横浜市青葉区奈良町で自動車破壊 ５月２８日から２９日,http://this.kiji.is/1169264637737632561?c=1347...,自動車破壊,（神奈川）横浜市青葉区奈良町で自動車破壊,５月２８日
277,2024/04/30,千葉県 松戸市（部品狙い）,（千葉）松戸市根本で部品狙い ４月２６日から２９日,http://this.kiji.is/1157892739379396851?c=1347...,部品狙い,（千葉）松戸市根本で部品狙い,４月２６日


In [12]:
# タイムゾーン取得
regex_tz = re.compile('(?<=日)(.*)')
def get_tz(s, regex):
    return regex.search(s).group()
    
data['tz'] = data['datetime'].apply(get_tz, regex = regex_tz)
data.head()

,date,title,title_detail,url,incident,content,datetime,tz
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,


In [13]:
# datetimeから日付を抽出
regex_date = re.compile('.+日')

def get_date(s, regex):
    return regex.search(s).group()

data['incident_date'] = data['datetime'].apply(get_date, regex = regex_date)
data.head()


,date,title,title_detail,url,incident,content,datetime,tz,incident_date
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,６月１３日
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,６月１３日
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,６月１３日
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,６月１４日
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,４月２７日


In [14]:
# incident_dateを半角に
data['incident_date'] = data['incident_date'].apply(lambda s: s.translate(zenkaku_table))
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date
0,2024/06/14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,6月13日
1,2024/06/14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,6月13日
2,2024/06/14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,6月13日
3,2024/06/14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,6月14日
4,2024/04/30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,4月27日


In [15]:
# year列を追加

data['date'] = pd.to_datetime(data['date'], format = '%Y/%m/%d')
data['year'] = data['date'].dt.year
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,6月13日,2024
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,6月13日,2024
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,6月13日,2024
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,6月14日,2024
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,4月27日,2024


In [16]:
# incident列からmonth列，day列を作成
data['incident_date'] = pd.to_datetime(data['incident_date'], format = '%m月%d日')
data['month'] = data['incident_date'].dt.month
data['day'] = data['incident_date'].dt.day
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,1900-06-13,2024,6,13
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,1900-06-13,2024,6,13
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,1900-06-13,2024,6,13
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,1900-06-14,2024,6,14
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,1900-04-27,2024,4,27


In [17]:
data[['year', 'month', 'day']] = data[['year', 'month', 'day']].astype(str)
data['incident_date'] = pd.to_datetime(data.year + '-' + data.month + '-' + data.day)
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,2024-06-13,2024,6,13
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,2024-06-13,2024,6,13
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,2024-06-13,2024,6,13
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,2024-06-14,2024,6,14
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,2024-04-27,2024,4,27


### 住所の加工

In [18]:
# 県名を取得
data['prefecture'] = data.content.apply(get_inside_par, regex = regex_par)
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day,prefecture
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,2024-06-13,2024,6,13,佐賀
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,2024-06-13,2024,6,13,長野
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,2024-06-13,2024,6,13,兵庫
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,2024-06-14,2024,6,14,群馬
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,2024-04-27,2024,4,27,広島


In [19]:
#「県」とか「府」とかをつける

def regular_prefecture(s):
    for s_reg in prefectures:
        if s in s_reg:
            return s_reg
        else:
            continue

data['prefecture'] = data['prefecture'].apply(regular_prefecture)
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day,prefecture
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,2024-06-13,2024,6,13,佐賀県
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,2024-06-13,2024,6,13,長野県
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,2024-06-13,2024,6,13,兵庫県
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,2024-06-14,2024,6,14,群馬県
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,2024-04-27,2024,4,27,広島県


In [20]:
def get_address_post(s):
    s = s.split('で')[0]
    res = re.sub(r'（.+）', '', s)
    return res                

In [21]:
data['address_post'] = data['content'].apply(get_address_post)
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day,prefecture,address_post
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,2024-06-13,2024,6,13,佐賀県,唐津市西唐津２丁目
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,2024-06-13,2024,6,13,長野県,木曽町福島
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,2024-06-13,2024,6,13,兵庫県,加古川市野口町長砂
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,2024-06-14,2024,6,14,群馬県,長野原町北軽井沢
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,2024-04-27,2024,4,27,広島県,広島市南区松原町


In [22]:
data['address'] = data['prefecture'] + data['address_post']
data.head()

,date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day,prefecture,address_post,address
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,2024-06-13,2024,6,13,佐賀県,唐津市西唐津２丁目,佐賀県唐津市西唐津２丁目
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,2024-06-13,2024,6,13,長野県,木曽町福島,長野県木曽町福島
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,2024-06-13,2024,6,13,兵庫県,加古川市野口町長砂,兵庫県加古川市野口町長砂
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,2024-06-14,2024,6,14,群馬県,長野原町北軽井沢,群馬県長野原町北軽井沢
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,2024-04-27,2024,4,27,広島県,広島市南区松原町,広島県広島市南区松原町


In [23]:

    
data.rename(columns = {'date': 'article_date'}, inplace = True)
data.head()

,article_date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day,prefecture,address_post,address
0,2024-06-14,佐賀県 唐津市（サル出没）,（佐賀）唐津市西唐津２丁目でサル出没 ６月１３日夜,http://this.kiji.is/1174237353901114287?c=1347...,サル出没,（佐賀）唐津市西唐津２丁目でサル出没,６月１３日夜,夜,2024-06-13,2024,6,13,佐賀県,唐津市西唐津２丁目,佐賀県唐津市西唐津２丁目
1,2024-06-14,長野県 木曽町（クマ出没）,（長野）木曽町福島でクマ出没 ６月１３日夕方,http://this.kiji.is/1174236381727195550?c=1347...,クマ出没,（長野）木曽町福島でクマ出没,６月１３日夕方,夕方,2024-06-13,2024,6,13,長野県,木曽町福島,長野県木曽町福島
2,2024-06-14,兵庫県 加古川市（声かけ）,（兵庫）加古川市野口町長砂で声かけ ６月１３日夕方,http://this.kiji.is/1174236184843403528?c=1347...,声かけ,（兵庫）加古川市野口町長砂で声かけ,６月１３日夕方,夕方,2024-06-13,2024,6,13,兵庫県,加古川市野口町長砂,兵庫県加古川市野口町長砂
3,2024-06-14,群馬県 長野原町（クマ出没）,（群馬）長野原町北軽井沢でクマ出没 ６月１４日朝,http://this.kiji.is/1174235855992815825?c=1347...,クマ出没,（群馬）長野原町北軽井沢でクマ出没,６月１４日朝,朝,2024-06-14,2024,6,14,群馬県,長野原町北軽井沢,群馬県長野原町北軽井沢
4,2024-04-30,広島県 広島市南区（暴行）,（広島）広島市南区松原町で暴行 ４月２７日［解決］,http://this.kiji.is/1158050512174973169?c=1347...,暴行,（広島）広島市南区松原町で暴行,４月２７日,,2024-04-27,2024,4,27,広島県,広島市南区松原町,広島県広島市南区松原町


In [24]:
# 動物系を除外
data['animal'] = data['incident'].apply(lambda s: True if ('出没' in s or '脱走' in s) else False)
data['animal'].value_counts()


animal
False    6866
True     3038
Name: count, dtype: int64

In [25]:
# 東京に限定．動物系を除外．
data = data.query('prefecture == "東京都" and animal == False')
data.head()

,article_date,title,title_detail,url,incident,content,datetime,tz,incident_date,year,month,day,prefecture,address_post,address,animal
89,2024-05-31,東京都 練馬区（下半身露出）,（東京）練馬区谷原５丁目で下半身露出 ５月３０日夕方,http://this.kiji.is/1169267542530458317?c=1347...,下半身露出,（東京）練馬区谷原５丁目で下半身露出,５月３０日夕方,夕方,2024-05-30,2024,5,30,東京都,練馬区谷原５丁目,東京都練馬区谷原５丁目,False
94,2024-04-30,東京都 小平市（暴行）,（東京）小平市回田町で暴行 ４月３０日夕方,http://this.kiji.is/1158030597921112344?c=1347...,暴行,（東京）小平市回田町で暴行,４月３０日夕方,夕方,2024-04-30,2024,4,30,東京都,小平市回田町,東京都小平市回田町,False
134,2024-05-31,東京都 北区（声かけ）,（東京）北区東十条２丁目で声かけ ５月３１日夕方,http://this.kiji.is/1169259575689561005?c=1347...,声かけ,（東京）北区東十条２丁目で声かけ,５月３１日夕方,夕方,2024-05-31,2024,5,31,東京都,北区東十条２丁目,東京都北区東十条２丁目,False
160,2024-04-30,東京都 日野市（つきまとい）,（東京）日野市旭が丘１丁目でつきまとい ４月３０日夕方,http://this.kiji.is/1158013460290633784?c=1347...,つきまとい,（東京）日野市旭が丘１丁目でつきまとい,４月３０日夕方,夕方,2024-04-30,2024,4,30,東京都,日野市旭が丘１丁目,東京都日野市旭が丘１丁目,False
163,2024-04-30,東京都 品川区（下半身露出）,（東京）品川区大崎１丁目で下半身露出 ４月３０日夕方,http://this.kiji.is/1158012909382320379?c=1347...,下半身露出,（東京）品川区大崎１丁目で下半身露出,４月３０日夕方,夕方,2024-04-30,2024,4,30,東京都,品川区大崎１丁目,東京都品川区大崎１丁目,False


In [26]:
data_save = data[['incident_date', 'tz', 'address', 'prefecture', 'incident', 'animal', 'title_detail', 'url']]
data_save.head()

,incident_date,tz,address,prefecture,incident,animal,title_detail,url
89,2024-05-30,夕方,東京都練馬区谷原５丁目,東京都,下半身露出,False,（東京）練馬区谷原５丁目で下半身露出 ５月３０日夕方,http://this.kiji.is/1169267542530458317?c=1347...
94,2024-04-30,夕方,東京都小平市回田町,東京都,暴行,False,（東京）小平市回田町で暴行 ４月３０日夕方,http://this.kiji.is/1158030597921112344?c=1347...
134,2024-05-31,夕方,東京都北区東十条２丁目,東京都,声かけ,False,（東京）北区東十条２丁目で声かけ ５月３１日夕方,http://this.kiji.is/1169259575689561005?c=1347...
160,2024-04-30,夕方,東京都日野市旭が丘１丁目,東京都,つきまとい,False,（東京）日野市旭が丘１丁目でつきまとい ４月３０日夕方,http://this.kiji.is/1158013460290633784?c=1347...
163,2024-04-30,夕方,東京都品川区大崎１丁目,東京都,下半身露出,False,（東京）品川区大崎１丁目で下半身露出 ４月３０日夕方,http://this.kiji.is/1158012909382320379?c=1347...


In [27]:
data_save.to_csv('data_processed.csv', index=False)